# Importing the libraries and Loading the data 

In [6]:
import sys
!{sys.executable} -m pip install openpyxl
import pandas as pd
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.feature_extraction import FeatureHasher
warnings.filterwarnings('ignore')

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /usr/local/bin/python3.12 -m pip install --upgrade pip


## File : Bitre_fatal_crash_dec2024.xlsx

In [7]:
file_path = 'bitre_fatal_crashes_dec2024.xlsx'
df_crash = pd.read_excel(file_path, sheet_name='BITRE_Fatal_Crash',header=4)

In [8]:
df_crash.columns

Index(['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type',
       'Number Fatalities', 'Bus \nInvolvement',
       'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement',
       'Speed Limit', 'National Remoteness Areas', 'SA4 Name 2021',
       'National LGA Name 2021', 'National Road Type', 'Christmas Period',
       'Easter Period', 'Day of week', 'Time of Day'],
      dtype='object')

# Data Cleansing And Matching 

In [9]:
df_crash.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,No,100,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,Weekday,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,No,80,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,Weekday,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,No,50,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,Weekday,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,No,100,Outer Regional Australia,New England and North West,Armidale Regional,National or State Highway,No,No,Weekday,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,-9,-9,-9,-9,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Day


In [10]:
# Trim column names to remove leading/trailing spaces
df_crash.columns = df_crash.columns.str.strip()

In [11]:
df_crash.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,No,100,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,Weekday,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,No,80,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,Weekday,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,No,50,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,Weekday,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,No,100,Outer Regional Australia,New England and North West,Armidale Regional,National or State Highway,No,No,Weekday,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,-9,-9,-9,-9,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Day


#### Now we check the column type by using a function 

In [12]:
def print_column_types(df):
    for i in df_crash.columns:
        if pd.api.types.is_numeric_dtype(df[i]):
            col_type = "numerical"
        else:
            col_type = "categorical"
        print(f"{i}: {col_type}")

In [13]:
print_column_types(df_crash)

Crash ID: numerical
State: categorical
Month: numerical
Year: numerical
Dayweek: categorical
Time: categorical
Crash Type: categorical
Number Fatalities: numerical
Bus 
Involvement: categorical
Heavy Rigid Truck Involvement: categorical
Articulated Truck Involvement: categorical
Speed Limit: categorical
National Remoteness Areas: categorical
SA4 Name 2021: categorical
National LGA Name 2021: categorical
National Road Type: categorical
Christmas Period: categorical
Easter Period: categorical
Day of week: categorical
Time of Day: categorical


#### Now we handle the missing data
We can see in the data that we have '-9' which signifies missing values , hence what we do is we change that to 'NA' for better handling of missing data 


In [14]:
df_crash.replace(-9,pd.NA,inplace=True)

In [15]:
df_crash.head(5)

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,No,100,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,Weekday,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,No,80,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,Weekday,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,No,50,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,Weekday,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,No,100,Outer Regional Australia,New England and North West,Armidale Regional,National or State Highway,No,No,Weekday,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,<NA>,<NA>,<NA>,<NA>,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Day


We know that the categorical columns are <br>
State: categorical <br>
Dayweek: categorical <br>
Time: categorical <br>
Crash Type: categorical <br>
Bus Involvement: categorical <br>
Heavy Rigid Truck Involvement: categorical <br>
Articulated Truck Involvement: categorical <br>
Speed Limit: categorical<br>
National Remoteness Areas: categorical<br>
SA4 Name 2021: categorical<br>
National LGA Name 2021: categorical<br>
National Road Type: categorical<br>
Christmas Period: categorical<br>
Easter Period: categorical<br>
Day of week: categorical<br>
Time of Day: categorical<br>

what we'll do is fill the values with unknown for now 

In [16]:
categorical_cols = [
    'State', 'Dayweek', 'Crash Type', 'Bus \nInvolvement', 
    'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement', 
    'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021', 
    'National Road Type', 'Christmas Period', 'Easter Period', 'Day of week', 'Time of Day'
]


In [17]:
for col in categorical_cols:
    df_crash[col] = df_crash[col].astype('category')
    if "Unknown" not in df_crash[col].cat.categories: ## Only add if it is missing
        df_crash[col] = df_crash[col].cat.add_categories("Unknown")
    df_crash[col].fillna("Unknown", inplace=True)

In [18]:
df_crash.tail()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
51279,19891246,NSW,1,1989,Wednesday,17:05:00,Single,1,Yes,Unknown,No,60,Unknown,Unknown,Unknown,Undetermined,No,No,Weekday,Day
51280,19892038,Vic,1,1989,Wednesday,18:50:00,Single,1,Yes,No,No,60,Unknown,Unknown,Unknown,Undetermined,No,No,Weekday,Night
51281,19894064,SA,1,1989,Wednesday,19:00:00,Multiple,1,No,Unknown,No,100,Unknown,Unknown,Unknown,Undetermined,No,No,Weekday,Night
51282,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,6,No,Unknown,Yes,100,Unknown,Unknown,Unknown,Undetermined,No,No,Weekday,Night
51283,19895133,WA,1,1989,Wednesday,21:00:00,Multiple,1,No,Unknown,No,<NA>,Unknown,Unknown,Unknown,Undetermined,No,No,Weekday,Night


Now for the numerical columns <br>
Crash ID: numerical<br>
Month: numerical<br>
Year: numerical<br>
Number Fatalities: numerical<br>

we can get the median and replace the NA values with it 

In [19]:
def clean_numeric_column(df, column_name):
    df[column_name] = df[column_name].astype(str)  # Convert to string
    df[column_name] = df[column_name].str.extract(r'(\d+)')  # Extract numbers
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')  # Convert to numeric

In [20]:
# Applying cleaning function to 'Speed Limit'
clean_numeric_column(df_crash, 'Speed Limit')

# Listing of numerical columns to clean and fill missing values
numerical_cols = ['Crash ID', 'Month', 'Year', 'Number Fatalities', 'Speed Limit']

# Filling missing numerical values with median after cleaning
for col in numerical_cols:
    df_crash[col].fillna(df_crash[col].median(), inplace=True)

In [21]:
df_crash.columns

Index(['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type',
       'Number Fatalities', 'Bus \nInvolvement',
       'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement',
       'Speed Limit', 'National Remoteness Areas', 'SA4 Name 2021',
       'National LGA Name 2021', 'National Road Type', 'Christmas Period',
       'Easter Period', 'Day of week', 'Time of Day'],
      dtype='object')

In [22]:
# Ensure Year and Month are integers
df_crash['Year'] = df_crash['Year'].astype(int)
df_crash['Month'] = df_crash['Month'].astype(int)

# Clean and format Time column
df_crash['Time'] = df_crash['Time'].astype(str).str.strip()
df_crash['Time'] = df_crash['Time'].apply(lambda x: x if ":" in x else "00:00:00")  # Fix missing times
df_crash['Time'] = pd.to_datetime(df_crash['Time'], format='%H:%M:%S', errors='coerce').dt.time  # Convert to proper format

# Create `Crash_Timestamp` combining Year, Month, and Time
df_crash['Crash_Timestamp'] = pd.to_datetime(df_crash[['Year', 'Month']].assign(DAY=1)) + pd.to_timedelta(df_crash['Time'].astype(str))

# Create `Crash_Date` (only date, for daily trends)
df_crash['Crash_Date'] = df_crash['Crash_Timestamp'].dt.date

# Create `Crash_Hour` (only hour, for hourly trends)
df_crash['Crash_Hour'] = df_crash['Crash_Timestamp'].dt.hour

# Drop redundant columns to avoid repetition
df_crash.drop(columns=['Year', 'Month', 'Time'], inplace=True)

In [23]:
df_crash.columns

Index(['Crash ID', 'State', 'Dayweek', 'Crash Type', 'Number Fatalities',
       'Bus \nInvolvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021',
       'National Road Type', 'Christmas Period', 'Easter Period',
       'Day of week', 'Time of Day', 'Crash_Timestamp', 'Crash_Date',
       'Crash_Hour'],
      dtype='object')

In [24]:
df_crash.rename(columns={'Bus \nInvolvement': 'Bus Involvement'}, inplace=True)

In [25]:
df_crash.columns

Index(['Crash ID', 'State', 'Dayweek', 'Crash Type', 'Number Fatalities',
       'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021',
       'National Road Type', 'Christmas Period', 'Easter Period',
       'Day of week', 'Time of Day', 'Crash_Timestamp', 'Crash_Date',
       'Crash_Hour'],
      dtype='object')

In [26]:
df_crash.drop(columns='Crash_Hour',inplace=True)

In [27]:
df_crash.columns

Index(['Crash ID', 'State', 'Dayweek', 'Crash Type', 'Number Fatalities',
       'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021',
       'National Road Type', 'Christmas Period', 'Easter Period',
       'Day of week', 'Time of Day', 'Crash_Timestamp', 'Crash_Date'],
      dtype='object')

In [28]:
# Ensure categorical fields can accommodate new values
df_crash['Bus Involvement'] = df_crash['Bus Involvement'].astype('category')
df_crash['Heavy Rigid Truck Involvement'] = df_crash['Heavy Rigid Truck Involvement'].astype('category')
df_crash['Articulated Truck Involvement'] = df_crash['Articulated Truck Involvement'].astype('category')

# Add 'Not Reported' as a category before filling NaNs
df_crash['Bus Involvement'] = df_crash['Bus Involvement'].cat.add_categories(['Not Reported'])
df_crash['Heavy Rigid Truck Involvement'] = df_crash['Heavy Rigid Truck Involvement'].cat.add_categories(['Not Reported'])
df_crash['Articulated Truck Involvement'] = df_crash['Articulated Truck Involvement'].cat.add_categories(['Not Reported'])

df_crash[['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']] = \
    df_crash[['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']].fillna("Not Reported")


In [29]:
df_crash.drop_duplicates(subset=['Crash ID'], keep='first', inplace=True)

In [30]:
df_crash.columns

Index(['Crash ID', 'State', 'Dayweek', 'Crash Type', 'Number Fatalities',
       'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021',
       'National Road Type', 'Christmas Period', 'Easter Period',
       'Day of week', 'Time of Day', 'Crash_Timestamp', 'Crash_Date'],
      dtype='object')

## File : Bitre_Fatalities_Dec_2024.xlsx

lets now understand the second file 

In [31]:
file_path_1 = 'bitre_fatalities_dec2024.xlsx'
df_fatality = pd.read_excel(file_path_1, sheet_name = 'BITRE_Fatality' ,header=4)
df_fatality.columns

Index(['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type',
       'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit', 'Road User', 'Gender',
       'Age', 'National Remoteness Areas', 'SA4 Name 2021',
       'National LGA Name 2021', 'National Road Type', 'Christmas Period',
       'Easter Period', 'Age Group', 'Day of week', 'Time of day'],
      dtype='object')

In [32]:
# Trim column names to remove leading/trailing spaces
df_fatality.columns = df_fatality.columns.str.strip()

In [33]:
if 'Unnamed: 0' in df_fatality.columns:
    df_fatality = df_fatality.drop(columns=['Unnamed: 0'])

In [34]:
df_fatality.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,...,Age,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,No,No,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,No,No,No,...,19,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,17_to_25,Weekday,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,No,No,No,...,33,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,26_to_39,Weekday,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,No,No,No,...,32,Outer Regional Australia,New England and North West,Armidale Regional,National or State Highway,No,No,26_to_39,Weekday,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,-9,-9,-9,...,62,Unknown,NaN,NaN,Undetermined,No,No,40_to_64,Weekday,Day


In [35]:
print(df_fatality.columns)

Index(['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type',
       'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit', 'Road User', 'Gender',
       'Age', 'National Remoteness Areas', 'SA4 Name 2021',
       'National LGA Name 2021', 'National Road Type', 'Christmas Period',
       'Easter Period', 'Age Group', 'Day of week', 'Time of day'],
      dtype='object')


In [36]:
def print_column_types(df):
    for i in df_fatality.columns:
        if pd.api.types.is_numeric_dtype(df[i]):
            col_type = "numerical"
        else:
            col_type = "categorical"
        print(f"{i}: {col_type}")

In [37]:
print_column_types(df_fatality)

Crash ID: numerical
State: categorical
Month: numerical
Year: numerical
Dayweek: categorical
Time: categorical
Crash Type: categorical
Bus Involvement: categorical
Heavy Rigid Truck Involvement: categorical
Articulated Truck Involvement: categorical
Speed Limit: categorical
Road User: categorical
Gender: categorical
Age: numerical
National Remoteness Areas: categorical
SA4 Name 2021: categorical
National LGA Name 2021: categorical
National Road Type: categorical
Christmas Period: categorical
Easter Period: categorical
Age Group: categorical
Day of week: categorical
Time of day: categorical


In [38]:
df_fatality.replace(-9,pd.NA,inplace=True)

In [39]:
for col in df_fatality.columns:
    if not pd.api.types.is_numeric_dtype(df_fatality[col]):
        df_fatality[col] = df_fatality[col].fillna("unknown")

In [40]:
# Get all numerical columns in the DataFrame
numeric_cols = df_fatality.select_dtypes(include=['number']).columns

# Iterate over numerical columns, compute the median and fill missing values
for col in numeric_cols:
    median_val = df_fatality[col].median()
    df_fatality[col] = df_fatality[col].fillna(median_val)

In [41]:
def clean_numeric_column(df, column_name):
    df[column_name] = df[column_name].astype(str)  # Convert to string
    df[column_name] = df[column_name].str.extract(r'(\d+)')  # Extract numbers
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')  # Convert to numeric

In [42]:
print_column_types(df_fatality)

Crash ID: numerical
State: categorical
Month: numerical
Year: numerical
Dayweek: categorical
Time: categorical
Crash Type: categorical
Bus Involvement: categorical
Heavy Rigid Truck Involvement: categorical
Articulated Truck Involvement: categorical
Speed Limit: categorical
Road User: categorical
Gender: categorical
Age: categorical
National Remoteness Areas: categorical
SA4 Name 2021: categorical
National LGA Name 2021: categorical
National Road Type: categorical
Christmas Period: categorical
Easter Period: categorical
Age Group: categorical
Day of week: categorical
Time of day: categorical


In [43]:
# Replace '<40' with 40 in the 'Speed Limit' column
df_fatality['Speed Limit'] = df_fatality['Speed Limit'].replace({'<40': 40})

# Optionally, convert the column to numeric if needed
df_fatality['Speed Limit'] = pd.to_numeric(df_fatality['Speed Limit'], errors='coerce')


In [44]:
# Applying cleaning function to 'Speed Limit'
clean_numeric_column(df_fatality, 'Speed Limit')

# Listing of numerical columns to clean and fill missing values
numerical_cols = ['Crash ID', 'Month', 'Year', 'Speed Limit']

# Filling missing numerical values with median after cleaning
for col in numerical_cols:
    df_fatality[col].fillna(df_fatality[col].median(), inplace=True)

In [45]:
df_fatality.to_excel('Partially_cleaned_data.xlsx',index = False)

In [46]:
# Find columns in df_fatality that are not in df_crash
missing_in_crash = set(df_fatality.columns) - set(df_crash.columns)
print("Columns in df_fatality but not in df_crash:", missing_in_crash)

Columns in df_fatality but not in df_crash: {'Gender', 'Road User', 'Age Group', 'Year', 'Month', 'Time of day', 'Age', 'Time'}


In [47]:

missing_cols = set(df_fatality.columns) - set(df_crash.columns)
# Directly assign the missing columns to df_crash (works only if row orders match)
for col in missing_cols:
    df_crash[col] = df_fatality[col]

# Verify the updated DataFrame columns
print("Updated df_crash columns:")
print(df_crash.columns)

Updated df_crash columns:
Index(['Crash ID', 'State', 'Dayweek', 'Crash Type', 'Number Fatalities',
       'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021',
       'National Road Type', 'Christmas Period', 'Easter Period',
       'Day of week', 'Time of Day', 'Crash_Timestamp', 'Crash_Date', 'Gender',
       'Road User', 'Age Group', 'Year', 'Month', 'Time of day', 'Age',
       'Time'],
      dtype='object')


In [48]:
df_final = pd.read_excel('Partially_cleaned.xlsx')

In [49]:
df_final.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,...,Christmas Period,Easter Period,Day of week,Time of Day,Bus Involvement,Age,Age Group,Gender,Road User,Time of day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,...,Yes,No,Weekday,Night,No,74,65_to_74,Male,Driver,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,...,No,No,Weekday,Day,No,19,17_to_25,Female,Driver,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,...,Yes,No,Weekday,Day,No,33,26_to_39,Female,Driver,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,...,No,No,Weekday,Day,No,32,26_to_39,Female,Driver,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,-9,-9,...,No,No,Weekday,Day,unknown,62,40_to_64,Male,Passenger,Day


In [50]:
df_final.drop_duplicates()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,...,Christmas Period,Easter Period,Day of week,Time of Day,Bus Involvement,Age,Age Group,Gender,Road User,Time of day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,...,Yes,No,Weekday,Night,No,74,65_to_74,Male,Driver,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,...,No,No,Weekday,Day,No,19,17_to_25,Female,Driver,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,...,Yes,No,Weekday,Day,No,33,26_to_39,Female,Driver,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,...,No,No,Weekday,Day,No,32,26_to_39,Female,Driver,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,-9,-9,...,No,No,Weekday,Day,unknown,62,40_to_64,Male,Passenger,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,19891246,NSW,1,1989,Wednesday,17:05:00,Single,1,Yes,-9,...,No,No,Weekday,Day,No,21,17_to_25,Male,Motorcycle rider,Night
51280,19892038,Vic,1,1989,Wednesday,18:50:00,Single,1,Yes,No,...,No,No,Weekday,Night,No,26,26_to_39,Male,Passenger,Night
51281,19894064,SA,1,1989,Wednesday,19:00:00,Multiple,1,No,-9,...,No,No,Weekday,Night,No,28,26_to_39,Male,Driver,Night
51282,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,6,No,-9,...,No,No,Weekday,Night,No,23,17_to_25,Female,Driver,Day


In [51]:
df_final.replace(-9,pd.NA,inplace=True)

In [52]:
df_final.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,...,Christmas Period,Easter Period,Day of week,Time of Day,Bus Involvement,Age,Age Group,Gender,Road User,Time of day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,...,Yes,No,Weekday,Night,No,74,65_to_74,Male,Driver,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,...,No,No,Weekday,Day,No,19,17_to_25,Female,Driver,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,...,Yes,No,Weekday,Day,No,33,26_to_39,Female,Driver,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,...,No,No,Weekday,Day,No,32,26_to_39,Female,Driver,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,<NA>,<NA>,...,No,No,Weekday,Day,unknown,62,40_to_64,Male,Passenger,Day


In [53]:
for col in df_final.columns:
    if pd.api.types.is_numeric_dtype(df_final[col]):
        median_value = df_final[col].median()
        df_final[col].fillna(median_value, inplace=True)
    else:
        mode_value = df_final[col].mode().iloc[0] if not df_final[col].mode().empty else ""
        df_final[col].fillna(mode_value, inplace=True)

In [54]:
df_final.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,...,Christmas Period,Easter Period,Day of week,Time of Day,Bus Involvement,Age,Age Group,Gender,Road User,Time of day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,...,Yes,No,Weekday,Night,No,74,65_to_74,Male,Driver,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,...,No,No,Weekday,Day,No,19,17_to_25,Female,Driver,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,...,Yes,No,Weekday,Day,No,33,26_to_39,Female,Driver,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,...,No,No,Weekday,Day,No,32,26_to_39,Female,Driver,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,No,No,...,No,No,Weekday,Day,unknown,62,40_to_64,Male,Passenger,Day


In [55]:
df_final = df_final.drop(columns=['Bus Involvement'])

In [56]:
df_final.rename(columns={'Bus \nInvolvement':'Bus Involvement'})

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus Involvement,Heavy Rigid Truck Involvement,...,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day,Age,Age Group,Gender,Road User,Time of day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,...,Arterial Road,Yes,No,Weekday,Night,74,65_to_74,Male,Driver,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,...,Local Road,No,No,Weekday,Day,19,17_to_25,Female,Driver,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,...,Local Road,Yes,No,Weekday,Day,33,26_to_39,Female,Driver,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,...,National or State Highway,No,No,Weekday,Day,32,26_to_39,Female,Driver,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,No,No,...,Undetermined,No,No,Weekday,Day,62,40_to_64,Male,Passenger,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,19891246,NSW,1,1989,Wednesday,17:05:00,Single,1,Yes,No,...,Undetermined,No,No,Weekday,Day,21,17_to_25,Male,Motorcycle rider,Night
51280,19892038,Vic,1,1989,Wednesday,18:50:00,Single,1,Yes,No,...,Undetermined,No,No,Weekday,Night,26,26_to_39,Male,Passenger,Night
51281,19894064,SA,1,1989,Wednesday,19:00:00,Multiple,1,No,No,...,Undetermined,No,No,Weekday,Night,28,26_to_39,Male,Driver,Night
51282,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,6,No,No,...,Undetermined,No,No,Weekday,Night,23,17_to_25,Female,Driver,Day


In [57]:
df_final['Speed Limit'] = df_final['Speed Limit'].replace({'<40': 40})

In [58]:
df_final['Speed Limit'].value_counts()

Speed Limit
100    18160
60     13682
80      6164
110     5576
50      3216
70      2596
90      1056
40       398
75       233
130      112
20        36
10        27
30        18
5          6
25         3
15         1
Name: count, dtype: int64

In [59]:
df_final['Age Group'].value_counts()

Age Group
40_to_64       13542
17_to_25       12745
26_to_39       11932
75_or_older     5213
65_to_74        4046
0_to_16         3702
unknown          104
Name: count, dtype: int64

In [60]:
mapping = {
    '0_to_16': '0-16',
    '17_to_25': '17-25',
    '26_to_39': '26-39',
    '40_to_64': '40-64',
    '65_to_74': '65-74',
    '75_or_older': '75+',
    'unknown': np.nan
}

In [61]:
df_final['Age Group'] = df_final['Age Group'].str.strip()

In [62]:
df_final['Age Group'] = df_final['Age Group'].replace(mapping)

In [63]:
age_order = ['0-16', '17-25', '26-39', '40-64', '65-74', '75+']
df_final['Age Group'] = pd.Categorical(df_final['Age Group'], categories=age_order, ordered=True)

In [64]:
df_final['Age Group'].value_counts()

Age Group
40-64    13542
17-25    12745
26-39    11932
75+       5213
65-74     4046
0-16      3702
Name: count, dtype: int64

In [65]:
last_column = df_final.columns[-1]
df_final = df_final.drop(columns=[last_column])

In [66]:
df_final.columns

Index(['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type',
       'Number Fatalities', 'Bus \nInvolvement',
       'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement',
       'Speed Limit', 'National Remoteness Areas', 'SA4 Name 2021',
       'National LGA Name 2021', 'National Road Type', 'Christmas Period',
       'Easter Period', 'Day of week', 'Time of Day', 'Age', 'Age Group',
       'Gender', 'Road User'],
      dtype='object')

# Adding the DWELL Count Data

In [67]:
excel_file_path = 'Cleaned_data_1.xlsx'

In [68]:
crash_df = pd.read_excel(excel_file_path)

In [71]:
csv_file_path = 'LGA (count of dwellings).csv'

In [72]:
with open(csv_file_path, "r", encoding="utf-8") as file:
    raw_lines = file.readlines()

In [73]:
data_lines = raw_lines[11:]
cleaned_data = []

for line in data_lines:
    parts = line.strip().split(',')
    if len(parts) >= 2:
        lga_name = parts[0].strip('"')
        try:
            count = int(parts[1])
            cleaned_data.append((lga_name, count))
        except ValueError:
            continue  # Skip rows with non-integer counts

In [74]:
lga_df = pd.DataFrame(cleaned_data, columns=["LGA", "Dwelling_Count"])

In [75]:
merged_df = crash_df.merge(
    lga_df, 
    left_on="National LGA Name 2021", 
    right_on="LGA", 
    how="left"
).drop(columns=["LGA"])

output_file_path = "Merged_Crash_Dwelling_Data.xlsx"
merged_df.to_excel(output_file_path, index=False)


### DATA Cleaning on the new file


In [85]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

def clean_data_column_by_column(file_path):
    """
    Clean the Merged Crash Dwelling Data Excel file column by column
    
    Parameters:
    file_path (str): Path to the Excel file
    
    Returns:
    pd.DataFrame: Cleaned dataframe
    """
    print(f"Reading data from {file_path}...")
    df = pd.read_excel(file_path)
    
    # Keep original data for comparison
    df_original = df.copy()
    
    print(f"Original shape: {df.shape}")
    print("\nStarting column-by-column cleaning process...")
    
    # Define unknown/missing value indicators
    unknown_values = ["Unknown", "unknown", "NA", "N/A", "NULL", "None", "#N/A", "-", ""]
    
    # ============== Step 1: Clean ID Columns ==============
    print("\n1. Cleaning ID Columns...")
    
    # Crash ID - check for uniqueness and ensure numeric
    print("  - Cleaning Crash ID...")
    df['Crash ID'] = pd.to_numeric(df['Crash ID'], errors='coerce')
    
    # Check for duplicates in Crash ID
    if df['Crash ID'].duplicated().any():
        print(f"  - Warning: {df['Crash ID'].duplicated().sum()} duplicate Crash IDs found")
    
    # ============== Step 2: Clean Location Columns ==============
    print("\n2. Cleaning Location Columns...")
    
    # State - standardize case and fix any inconsistencies
    print("  - Cleaning State...")
    df['State'] = df['State'].str.strip()
    
    # Check for any invalid states
    valid_states = ['NSW', 'Vic', 'Qld', 'SA', 'WA', 'Tas', 'NT', 'ACT']
    invalid_states = df[~df['State'].isin(valid_states)]['State'].unique()
    if len(invalid_states) > 0:
        print(f"  - Warning: Invalid states found: {invalid_states}")
        # Map any common misspellings
        state_mapping = {
            'VIC': 'Vic', 
            'QLD': 'Qld', 
            'TAS': 'Tas'
        }
        df['State'] = df['State'].replace(state_mapping)
    
    # National Remoteness Areas - replace 'Unknown' with NaN
    print("  - Cleaning National Remoteness Areas...")
    df['National Remoteness Areas'] = df['National Remoteness Areas'].replace('Unknown', np.nan)
    
    # SA4 Name 2021 - replace 'Unknown' with NaN
    print("  - Cleaning SA4 Name 2021...")
    df['SA4 Name 2021'] = df['SA4 Name 2021'].replace('Unknown', np.nan)
    
    # National LGA Name 2021 - replace 'Unknown' with NaN
    print("  - Cleaning National LGA Name 2021...")
    df['National LGA Name 2021'] = df['National LGA Name 2021'].replace('Unknown', np.nan)
    
    # National Road Type - replace 'Undetermined' with NaN if needed
    print("  - Cleaning National Road Type...")
    # Only replace Undetermined if specifically requested
    # df['National Road Type'] = df['National Road Type'].replace('Undetermined', np.nan)
    
    # ============== Step 3: Clean Time-Related Columns ==============
    print("\n3. Cleaning Time-Related Columns...")
    
    # Month - ensure it's an integer between 1-12
    print("  - Cleaning Month...")
    df['Month'] = pd.to_numeric(df['Month'], errors='coerce')
    invalid_months = df[(df['Month'] < 1) | (df['Month'] > 12)].index
    if len(invalid_months) > 0:
        print(f"  - Fixing {len(invalid_months)} invalid month values")
        # Impute invalid months with the median
        df.loc[invalid_months, 'Month'] = df['Month'].median()
    
    # Year - ensure it's a valid year
    print("  - Cleaning Year...")
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    min_year = 1989  # Based on data analysis
    max_year = 2024  # Based on data analysis
    invalid_years = df[(df['Year'] < min_year) | (df['Year'] > max_year)].index
    if len(invalid_years) > 0:
        print(f"  - Fixing {len(invalid_years)} invalid year values")
        # Impute invalid years with the median
        df.loc[invalid_years, 'Year'] = df['Year'].median()
    
    # Time - ensure proper time format
    print("  - Cleaning Time...")
    # Convert to proper time format if needed
    try:
        df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce').dt.time
    except:
        print("  - Warning: Time column could not be converted to proper time format")
    
    # Extract hour for analysis and imputation
    df['Hour'] = df['Time'].apply(lambda x: int(str(x).split(':')[0]) if pd.notna(x) else np.nan)
    
    # Day of week - replace 'Unknown' with NaN
    print("  - Cleaning Day of week...")
    df['Day of week'] = df['Day of week'].replace('Unknown', np.nan)
    
    # Time of Day - replace 'Unknown' with NaN
    print("  - Cleaning Time of Day...")
    df['Time of Day'] = df['Time of Day'].replace('Unknown', np.nan)
    
    # Dayweek - validate values
    print("  - Cleaning Dayweek...")
    valid_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    invalid_days = df[~df['Dayweek'].isin(valid_days)]['Dayweek'].unique()
    if len(invalid_days) > 0:
        print(f"  - Warning: Invalid days found: {invalid_days}")
        # Could map common misspellings here if needed
    
    # Period indicators
    print("  - Cleaning Christmas/Easter Period...")
    for col in ['Christmas Period', 'Easter Period']:
        # Ensure values are Yes or No
        invalid_values = df[~df[col].isin(['Yes', 'No'])][col].unique()
        if len(invalid_values) > 0:
            print(f"  - Warning: Invalid values in {col}: {invalid_values}")
            # Could correct common errors here
    
    # ============== Step 4: Clean Crash Information Columns ==============
    print("\n4. Cleaning Crash Information Columns...")
    
    # Crash Type
    print("  - Cleaning Crash Type...")
    valid_crash_types = ['Single', 'Multiple']
    invalid_crash_types = df[~df['Crash Type'].isin(valid_crash_types)]['Crash Type'].unique()
    if len(invalid_crash_types) > 0:
        print(f"  - Warning: Invalid crash types found: {invalid_crash_types}")
        # Could map common values here
    
    # Number Fatalities - ensure numeric and positive
    print("  - Cleaning Number Fatalities...")
    df['Number Fatalities'] = pd.to_numeric(df['Number Fatalities'], errors='coerce')
    invalid_fatalities = df[df['Number Fatalities'] < 1].index
    if len(invalid_fatalities) > 0:
        print(f"  - Fixing {len(invalid_fatalities)} invalid fatality counts")
        # Set invalid counts to 1 (minimum reasonable value)
        df.loc[invalid_fatalities, 'Number Fatalities'] = 1
    
    # Speed Limit - ensure numeric and reasonable
    print("  - Cleaning Speed Limit...")
    df['Speed Limit'] = pd.to_numeric(df['Speed Limit'], errors='coerce')
    # Define reasonable speed limits (e.g., 5-130 based on data)
    min_speed = 5
    max_speed = 130
    invalid_speeds = df[(df['Speed Limit'] < min_speed) | (df['Speed Limit'] > max_speed)].index
    if len(invalid_speeds) > 0:
        print(f"  - Fixing {len(invalid_speeds)} invalid speed limits")
        # Impute invalid speeds with the median
        df.loc[invalid_speeds, 'Speed Limit'] = df['Speed Limit'].median()
    
    # Vehicle involvement columns
    print("  - Cleaning vehicle involvement columns...")
    involvement_cols = [
        'Bus \nInvolvement', 
        'Heavy Rigid Truck Involvement', 
        'Articulated Truck Involvement'
    ]
    
    for col in involvement_cols:
        invalid_values = df[~df[col].isin(['Yes', 'No'])][col].unique()
        if len(invalid_values) > 0:
            print(f"  - Warning: Invalid values in {col}: {invalid_values}")
            # Replace with most common value (No)
            df[col] = df[col].replace(invalid_values, 'No')
    
    # ============== Step 5: Clean Person Information Columns ==============
    print("\n5. Cleaning Person Information Columns...")
    
    # Age - ensure numeric and reasonable
    print("  - Cleaning Age...")
    df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
    df['Age'] = df['Age'].replace('Unknown', np.nan)
    
    # Define reasonable age limits
    min_age = 0
    max_age = 101  # Based on data analysis
    invalid_ages = df[(df['Age'] < min_age) | (df['Age'] > max_age)].index
    if len(invalid_ages) > 0:
        print(f"  - Fixing {len(invalid_ages)} invalid age values")
        # Impute invalid ages with the median
        df.loc[invalid_ages, 'Age'] = df['Age'].median()
    
    # Age Group - handle empty values
    print("  - Cleaning Age Group...")
    df['Age Group'] = df['Age Group'].replace('', np.nan)
    
    # Gender - standardize and handle 'unknown'
    print("  - Cleaning Gender...")
    df['Gender'] = df['Gender'].replace('unknown', np.nan)
    
    # Road User - handle 'Unknown'
    print("  - Cleaning Road User...")
    df['Road User'] = df['Road User'].replace('Unknown', np.nan)
    
    # ============== Step 6: Clean Dwelling Count ==============
    print("\n6. Cleaning Dwelling Count...")
    
    # Dwelling_Count - handle empty values
    df['Dwelling_Count'] = pd.to_numeric(df['Dwelling_Count'].replace('', np.nan), errors='coerce')
    
    # ============== Step 7: Imputation - Column by Column ==============
    print("\n7. Starting column-by-column imputation...")
    
    # 7.1 First, impute simple categorical columns with their mode
    print("  - Imputing simple categorical columns...")
    categorical_simple = [
        'State', 
        'Dayweek', 
        'Crash Type', 
        'Bus \nInvolvement', 
        'Heavy Rigid Truck Involvement', 
        'Articulated Truck Involvement',
        'Christmas Period', 
        'Easter Period', 
        'Gender'
    ]
    
    for col in categorical_simple:
        if df[col].isna().sum() > 0:
            mode_value = df[col].mode()[0]
            print(f"  - Imputing {df[col].isna().sum()} missing values in {col} with mode: {mode_value}")
            df[col].fillna(mode_value, inplace=True)
    
    # 7.2 Impute Day of week based on Dayweek
    print("  - Imputing Day of week based on Dayweek...")
    day_map = {
        'Monday': 'Weekday',
        'Tuesday': 'Weekday',
        'Wednesday': 'Weekday',
        'Thursday': 'Weekday',
        'Friday': 'Weekday',
        'Saturday': 'Weekend',
        'Sunday': 'Weekend'
    }
    
    if df['Day of week'].isna().sum() > 0:
        # Create a mapping based on Dayweek
        df.loc[df['Day of week'].isna(), 'Day of week'] = df.loc[df['Day of week'].isna(), 'Dayweek'].map(day_map)
        
        # If any still missing, use mode
        if df['Day of week'].isna().sum() > 0:
            mode_value = df['Day of week'].mode()[0]
            print(f"  - Imputing remaining {df['Day of week'].isna().sum()} missing values in Day of week with mode: {mode_value}")
            df['Day of week'].fillna(mode_value, inplace=True)
    
    # 7.3 Impute Time of Day based on Hour
    print("  - Imputing Time of Day based on Hour...")
    def hour_to_time_of_day(hour):
        if pd.isna(hour):
            return np.nan
        return 'Day' if 6 <= hour < 18 else 'Night'
    
    if df['Time of Day'].isna().sum() > 0:
        df.loc[df['Time of Day'].isna(), 'Time of Day'] = df.loc[df['Time of Day'].isna(), 'Hour'].apply(hour_to_time_of_day)
        
        # If any still missing, use mode
        if df['Time of Day'].isna().sum() > 0:
            mode_value = df['Time of Day'].mode()[0]
            print(f"  - Imputing remaining {df['Time of Day'].isna().sum()} missing values in Time of Day with mode: {mode_value}")
            df['Time of Day'].fillna(mode_value, inplace=True)
    
    # 7.4 Impute Age Group based on Age
    print("  - Imputing Age Group based on Age...")
    def age_to_group(age):
        if pd.isna(age):
            return np.nan
        age = int(age)
        if age <= 16:
            return '0-16'
        elif age <= 25:
            return '17-25'
        elif age <= 39:
            return '26-39'
        elif age <= 64:
            return '40-64'
        elif age <= 74:
            return '65-74'
        else:
            return '75+'
    
    if df['Age Group'].isna().sum() > 0:
        df.loc[df['Age Group'].isna(), 'Age Group'] = df.loc[df['Age Group'].isna(), 'Age'].apply(age_to_group)
        
        # If any still missing, use mode
        if df['Age Group'].isna().sum() > 0:
            mode_value = df['Age Group'].mode()[0]
            print(f"  - Imputing remaining {df['Age Group'].isna().sum()} missing values in Age Group with mode: {mode_value}")
            df['Age Group'].fillna(mode_value, inplace=True)
    
    # 7.5 Impute Road User with Random Forest
    print("  - Imputing Road User...")
    if df['Road User'].isna().sum() > 0:
        # Features for imputation
        features = ['Age', 'Gender', 'Crash Type', 'Speed Limit']
        
        # Prepare the data
        X_train = df.loc[~df['Road User'].isna(), features].copy()
        y_train = df.loc[~df['Road User'].isna(), 'Road User']
        X_missing = df.loc[df['Road User'].isna(), features].copy()
        
        # Handle categorical features
        for feat in X_train.select_dtypes(include=['object']).columns:
            # Create categorical encoders
            X_train_categories = pd.Categorical(X_train[feat])
            
            # Convert to codes for training data
            X_train[feat] = X_train_categories.codes
            
            # For missing data, ensure we use same categories 
            # Create a mapping dictionary from values to codes
            category_map = {category: code for code, category in enumerate(X_train_categories.categories)}
            
            # Map the values in X_missing to codes
            X_missing[feat] = X_missing[feat].map(category_map)
            
            # Handle any values in X_missing that weren't in X_train
            if X_missing[feat].isna().any():
                # Use the most common code as fallback
                most_common_code = pd.Series(X_train[feat]).mode()[0]
                X_missing[feat].fillna(most_common_code, inplace=True)
        
        # Fill any missing values in features
        for col in X_train.columns:
            if X_train[col].isna().sum() > 0:
                X_train[col].fillna(X_train[col].median() if X_train[col].dtype.kind in 'ifc' else X_train[col].mode()[0], inplace=True)
            if X_missing[col].isna().sum() > 0:
                X_missing[col].fillna(X_train[col].median() if X_train[col].dtype.kind in 'ifc' else X_train[col].mode()[0], inplace=True)
        
        # Train a Random Forest classifier
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        
        # Predict missing values
        missing_indices = df.loc[df['Road User'].isna()].index
        df.loc[missing_indices, 'Road User'] = rf.predict(X_missing)
        
        print(f"  - Imputed {len(missing_indices)} values in Road User using Random Forest")
    
    # 7.6 Impute complex location columns
    # These columns have high missingness (>80%) so we'll use a targeted approach
    
    # 7.6.1 Impute National Remoteness Areas based on State and other location info
    print("  - Imputing National Remoteness Areas...")
    if df['National Remoteness Areas'].isna().sum() > 0:
        # Use State and Road Type to predict remoteness area
        features = ['State', 'National Road Type', 'Speed Limit']
        
        # Prepare the data
        X_train = df.loc[~df['National Remoteness Areas'].isna(), features].copy()
        y_train = df.loc[~df['National Remoteness Areas'].isna(), 'National Remoteness Areas']
        X_missing = df.loc[df['National Remoteness Areas'].isna(), features].copy()
        
        # Handle categorical features
        for feat in X_train.select_dtypes(include=['object']).columns:
            # Create categorical encoders
            X_train_categories = pd.Categorical(X_train[feat])
            
            # Convert to codes for training data
            X_train[feat] = X_train_categories.codes
            
            # For missing data, ensure we use same categories 
            # Create a mapping dictionary from values to codes
            category_map = {category: code for code, category in enumerate(X_train_categories.categories)}
            
            # Map the values in X_missing to codes
            X_missing[feat] = X_missing[feat].map(category_map)
            
            # Handle any values in X_missing that weren't in X_train
            if X_missing[feat].isna().any():
                # Use the most common code as fallback
                most_common_code = pd.Series(X_train[feat]).mode()[0]
                X_missing[feat].fillna(most_common_code, inplace=True)
        
        # Fill any missing values in features
        for col in X_train.columns:
            if X_train[col].isna().sum() > 0:
                X_train[col].fillna(X_train[col].median() if X_train[col].dtype.kind in 'ifc' else X_train[col].mode()[0], inplace=True)
            if X_missing[col].isna().sum() > 0:
                X_missing[col].fillna(X_train[col].median() if X_train[col].dtype.kind in 'ifc' else X_train[col].mode()[0], inplace=True)
        
        # Train a Random Forest classifier
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        
        # Predict missing values
        missing_indices = df.loc[df['National Remoteness Areas'].isna()].index
        df.loc[missing_indices, 'National Remoteness Areas'] = rf.predict(X_missing)
        
        print(f"  - Imputed {len(missing_indices)} values in National Remoteness Areas using Random Forest")
    
    # 7.6.2 Impute SA4 Name 2021 based on State and Remoteness
    print("  - Imputing SA4 Name 2021...")
    if df['SA4 Name 2021'].isna().sum() > 0:
        # For simplicity in this high-missing scenario, we'll create a "State-Region" mapping
        # Create a combined feature from State and Remoteness
        df['State_Region'] = df['State'] + '_' + df['National Remoteness Areas'].astype(str)
        
        # Group by State_Region and count SA4 occurrences
        sa4_by_region = df.loc[~df['SA4 Name 2021'].isna(), ['State_Region', 'SA4 Name 2021']].groupby('State_Region')['SA4 Name 2021'].agg(lambda x: x.value_counts().index[0]).to_dict()
        
        # Apply the mapping to fill missing values
        df.loc[df['SA4 Name 2021'].isna(), 'SA4 Name 2021'] = df.loc[df['SA4 Name 2021'].isna(), 'State_Region'].map(sa4_by_region)
        
        # For any remaining missing values, use most common by State
        sa4_by_state = df.loc[~df['SA4 Name 2021'].isna(), ['State', 'SA4 Name 2021']].groupby('State')['SA4 Name 2021'].agg(lambda x: x.value_counts().index[0]).to_dict()
        
        still_missing = df['SA4 Name 2021'].isna()
        if still_missing.sum() > 0:
            df.loc[still_missing, 'SA4 Name 2021'] = df.loc[still_missing, 'State'].map(sa4_by_state)
            
            # If any still missing, use overall mode
            if df['SA4 Name 2021'].isna().sum() > 0:
                mode_value = df.loc[~df['SA4 Name 2021'].isna(), 'SA4 Name 2021'].mode()[0]
                df['SA4 Name 2021'].fillna(mode_value, inplace=True)
        
        # Remove the temporary column
        df.drop('State_Region', axis=1, inplace=True)
        
        print(f"  - Imputed {still_missing.sum()} values in SA4 Name 2021 using region mapping")
    
    # 7.6.3 Impute National LGA Name 2021 based on SA4 and State
    print("  - Imputing National LGA Name 2021...")
    if df['National LGA Name 2021'].isna().sum() > 0:
        # Create mapping from SA4 to most common LGA
        lga_by_sa4 = df.loc[~df['National LGA Name 2021'].isna(), ['SA4 Name 2021', 'National LGA Name 2021']].groupby('SA4 Name 2021')['National LGA Name 2021'].agg(lambda x: x.value_counts().index[0]).to_dict()
        
        # Apply the mapping to fill missing values
        df.loc[df['National LGA Name 2021'].isna(), 'National LGA Name 2021'] = df.loc[df['National LGA Name 2021'].isna(), 'SA4 Name 2021'].map(lga_by_sa4)
        
        # For any remaining missing values, use most common by State
        still_missing = df['National LGA Name 2021'].isna()
        if still_missing.sum() > 0:
            lga_by_state = df.loc[~df['National LGA Name 2021'].isna(), ['State', 'National LGA Name 2021']].groupby('State')['National LGA Name 2021'].agg(lambda x: x.value_counts().index[0]).to_dict()
            
            df.loc[still_missing, 'National LGA Name 2021'] = df.loc[still_missing, 'State'].map(lga_by_state)
            
            # If any still missing, use overall mode
            if df['National LGA Name 2021'].isna().sum() > 0:
                mode_value = df.loc[~df['National LGA Name 2021'].isna(), 'National LGA Name 2021'].mode()[0]
                df['National LGA Name 2021'].fillna(mode_value, inplace=True)
        
        print(f"  - Imputed {still_missing.sum()} values in National LGA Name 2021 using SA4 mapping")
    
    # 7.7 Impute Dwelling_Count using Random Forest
    print("  - Imputing Dwelling_Count...")
    if df['Dwelling_Count'].isna().sum() > 0:
        # Features for imputation
        features = ['State', 'National LGA Name 2021', 'National Remoteness Areas', 'Year']
        
        # Prepare the data
        X_train = df.loc[~df['Dwelling_Count'].isna(), features].copy()
        y_train = df.loc[~df['Dwelling_Count'].isna(), 'Dwelling_Count']
        X_missing = df.loc[df['Dwelling_Count'].isna(), features].copy()
        
        # Handle categorical features
        for feat in X_train.select_dtypes(include=['object']).columns:
            # Create categorical encoders
            X_train_categories = pd.Categorical(X_train[feat])
            
            # Convert to codes for training data
            X_train[feat] = X_train_categories.codes
            
            # For missing data, ensure we use same categories 
            # Create a mapping dictionary from values to codes
            category_map = {category: code for code, category in enumerate(X_train_categories.categories)}
            
            # Map the values in X_missing to codes
            X_missing[feat] = X_missing[feat].map(category_map)
            
            # Handle any values in X_missing that weren't in X_train
            if X_missing[feat].isna().any():
                # Use the most common code as fallback
                most_common_code = pd.Series(X_train[feat]).mode()[0]
                X_missing[feat].fillna(most_common_code, inplace=True)
            
        # Fill any missing values in features
        for col in X_train.columns:
            if X_train[col].isna().sum() > 0:
                if X_train[col].dtype.kind in 'ifc':
                    X_train[col].fillna(X_train[col].median(), inplace=True)
                else:
                    X_train[col].fillna(X_train[col].mode()[0], inplace=True)
                    
            if X_missing[col].isna().sum() > 0:
                if X_train[col].dtype.kind in 'ifc':
                    X_missing[col].fillna(X_train[col].median(), inplace=True)
                else:
                    X_missing[col].fillna(X_train[col].mode()[0], inplace=True)
        
        # Train a Random Forest regressor
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        
        # Predict missing values
        missing_indices = df.loc[df['Dwelling_Count'].isna()].index
        predictions = rf.predict(X_missing)
        
        # Round to integers (dwelling counts should be whole numbers)
        predictions = np.round(predictions).astype(int)
        
        # Ensure no negative values
        predictions = np.maximum(0, predictions)
        
        df.loc[missing_indices, 'Dwelling_Count'] = predictions
        
        print(f"  - Imputed {len(missing_indices)} values in Dwelling_Count using Random Forest")
    
    # 7.8 Handle any remaining missing values
    print("\n8. Final check for missing values...")
    remaining_missing = df.isna().sum()
    cols_with_missing = remaining_missing[remaining_missing > 0].index.tolist()
    
    if cols_with_missing:
        print(f"  - Columns still having missing values: {cols_with_missing}")
        
        for col in cols_with_missing:
            missing_count = df[col].isna().sum()
            
            if df[col].dtype.kind in 'ifc':  # If numeric
                # Use median imputation for any remaining numeric missing values
                median_val = df[col].median()
                df[col].fillna(median_val, inplace=True)
                print(f"  - Imputed {missing_count} remaining missing values in {col} with median: {median_val}")
            else:  # If categorical/object
                # Use mode imputation for any remaining categorical missing values
                mode_val = df[col].mode()[0]
                df[col].fillna(mode_val, inplace=True)
                print(f"  - Imputed {missing_count} remaining missing values in {col} with mode: {mode_val}")
    else:
        print("  - No remaining missing values!")
    
    # ============== Step 8: Drop temporary columns ==============
    print("\n9. Cleaning up temporary columns...")
    # Drop the Hour column if it was created
    if 'Hour' in df.columns:
        df.drop('Hour', axis=1, inplace=True)
    
    # ============== Step 9: Validate Data Types ==============
    print("\n10. Validating data types...")
    
    # Ensure numeric columns are properly typed
    numeric_cols = ['Crash ID', 'Month', 'Year', 'Number Fatalities', 'Speed Limit', 'Age', 'Dwelling_Count']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # If any values were coerced to NaN, fix them
            if df[col].isna().sum() > 0:
                print(f"  - Warning: {df[col].isna().sum()} values in {col} couldn't be converted to numeric")
                df[col].fillna(df[col].median(), inplace=True)
    
    # ============== Step 10: Generate Data Quality Report ==============
    print("\n11. Generating data quality report...")
    
    # Compare original and cleaned data
    comparison = pd.DataFrame({
        'Column': df.columns,
        'Original_Missing': [df_original[col].isna().sum() + 
                            (df_original[col] == 'Unknown').sum() if df_original[col].dtype == 'object' else 
                            df_original[col].isna().sum() 
                            for col in df.columns],
        'Cleaned_Missing': [df[col].isna().sum() for col in df.columns],
        'Values_Imputed': [
            (df_original[col].isna().sum() + 
            (df_original[col] == 'Unknown').sum() if df_original[col].dtype == 'object' else 
            df_original[col].isna().sum()) - df[col].isna().sum() 
            for col in df.columns
        ]
    })
    
    # Calculate imputation effectiveness
    if comparison['Original_Missing'].sum() > 0:
        imputation_effectiveness = (1 - comparison['Cleaned_Missing'].sum() / comparison['Original_Missing'].sum()) * 100
    else:
        imputation_effectiveness = 100
    
    print(f"\nImputation Effectiveness: {imputation_effectiveness:.2f}%")
    print(f"Total values imputed: {comparison['Values_Imputed'].sum()}")
    
    # Print top columns with most imputations
    top_imputed = comparison.sort_values('Values_Imputed', ascending=False).head(5)
    print("\nTop 5 columns with most imputations:")
    for _, row in top_imputed.iterrows():
        print(f"  - {row['Column']}: {row['Values_Imputed']} values imputed")
    
    return df, comparison



In [87]:
if __name__ == "__main__":
    file_path = "Merged_Crash_Dwelling_Data.xlsx"
    cleaned_df, comparison_report = clean_data_column_by_column(file_path)
    
    # Save the cleaned data
    cleaned_df.to_excel("Cleaned_Crash_Dwelling_Data.xlsx", index=False)
    
    
    print("\nCleaned data saved to Cleaned_Crash_Dwelling_Data.xlsx")


Reading data from Merged_Crash_Dwelling_Data.xlsx...
Original shape: (51284, 25)

Starting column-by-column cleaning process...

1. Cleaning ID Columns...
  - Cleaning Crash ID...

2. Cleaning Location Columns...
  - Cleaning State...
  - Cleaning National Remoteness Areas...
  - Cleaning SA4 Name 2021...
  - Cleaning National LGA Name 2021...
  - Cleaning National Road Type...

3. Cleaning Time-Related Columns...
  - Cleaning Month...
  - Cleaning Year...
  - Cleaning Time...
  - Cleaning Day of week...
  - Cleaning Time of Day...
  - Cleaning Dayweek...
  - Cleaning Christmas/Easter Period...

4. Cleaning Crash Information Columns...
  - Cleaning Crash Type...
  - Cleaning Number Fatalities...
  - Cleaning Speed Limit...
  - Cleaning vehicle involvement columns...

5. Cleaning Person Information Columns...
  - Cleaning Age...
  - Cleaning Age Group...
  - Cleaning Gender...
  - Cleaning Road User...

6. Cleaning Dwelling Count...

7. Starting column-by-column imputation...
  - Imputi

In [91]:
for col in cleaned_df:
    print(cleaned_df[col].value_counts())

Crash ID
20241115    1
19981448    1
19984015    1
19984027    1
19984134    1
           ..
20093129    1
20091304    1
20091143    1
20095098    1
19895133    1
Name: count, Length: 51284, dtype: int64
State
NSW    15610
Vic    11293
Qld    10304
WA      6213
SA      4342
NT      1554
Tas     1497
ACT      471
Name: count, dtype: int64
Month
12    4601
3     4562
10    4424
11    4350
5     4350
8     4269
7     4215
9     4199
6     4153
4     4139
1     4123
2     3899
Name: count, dtype: int64
Year
1989    2407
1990    2050
1991    1874
1995    1822
1996    1768
1993    1737
1992    1736
1994    1702
2000    1628
1997    1601
2001    1584
1998    1573
1999    1553
2002    1525
2005    1472
2007    1453
2006    1452
2003    1445
2004    1444
2009    1347
2008    1315
2010    1233
2016    1200
2024    1191
2012    1190
2011    1151
2023    1149
2017    1127
2022    1103
2015    1102
2013    1100
2019    1098
2018    1055
2014    1050
2021    1049
2020     998
Name: count, dtype: int

## Adding the GeoJson files to the cleaned data 

In [2]:
!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00


In [6]:
import sys
!{sys.executable} -m pip install geopandas
import geopandas as gpd
import pandas as pd

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.10.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (5.5 kB)
  Using cached pyproj-3.7.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (31 kB)
  Using cached shapely-2.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached pyogrio-0.10.0-cp312-cp312-macosx_12_0_arm64.whl (15.1 MB)
Using cached pyproj-3.7.1-cp312-cp312-macosx_14_0_arm64.whl (4.7 MB)
Using cached shapely-2.1.0-cp312-cp312-macosx_11_0_arm64.whl (1.6 MB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /usr/local/bin/python3.12 -m pip install --upgrade pip


In [13]:
import geopandas as gpd
import pandas as pd

# Load LGA GeoJSON
lga = gpd.read_file("GJFiles/LGA_2021_AUST_GDA94.geojson")

# Reproject to a projected CRS for accurate centroids
lga = lga.to_crs("EPSG:3577")  # GDA94 / Australian Albers

# Calculate centroids and convert back to lat/lon
lga['longitude'] = lga.geometry.centroid.to_crs(epsg=4326).x
lga['latitude'] = lga.geometry.centroid.to_crs(epsg=4326).y

# Keep only relevant columns
lga_coords = lga[['LGA_NAME21', 'latitude', 'longitude']]

# Load cleaned Excel data
df_clean = pd.read_excel("Cleaned_Crash_Dwelling_Data.xlsx")

# Merge using LGA name
df_merged = df_clean.merge(lga_coords, left_on='National LGA Name 2021', right_on='LGA_NAME21', how='left')

# Drop duplicate column
df_merged = df_merged.drop(columns=['LGA_NAME21'])

# Save to Excel
output_path = "Cleaned_Data_with_Coordinates.xlsx"
df_merged.to_excel(output_path, index=False)

print(f"✅ Final file saved to: {output_path}")


✅ Final file saved to: Cleaned_Data_with_Coordinates.xlsx


In [14]:
import geopandas as gpd
import pandas as pd

# Load Excel with missing lat/longs
df = pd.read_excel("Cleaned_Data_with_Coordinates.xlsx")

# Load LGA GeoJSON again
lga = gpd.read_file("GJFiles/LGA_2021_AUST_GDA94.geojson")
lga = lga.to_crs("EPSG:3577")
lga['longitude'] = lga.geometry.centroid.to_crs(epsg=4326).x
lga['latitude'] = lga.geometry.centroid.to_crs(epsg=4326).y

# Create lookup dict from LGA name to coordinates
lga_lookup = lga.set_index('LGA_NAME21')[['latitude', 'longitude']].to_dict(orient='index')

# Function to fill missing values using LGA name
def fill_coords(row):
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):
        match = lga_lookup.get(row['National LGA Name 2021'])
        if match:
            row['latitude'] = match['latitude']
            row['longitude'] = match['longitude']
    return row

# Apply coordinate fill
df = df.apply(fill_coords, axis=1)

# Fill any remaining with state-level averages
df['latitude'] = df.groupby('State')['latitude'].transform(lambda x: x.fillna(x.mean()))
df['longitude'] = df.groupby('State')['longitude'].transform(lambda x: x.fillna(x.mean()))

# Save imputed file
output_path = "Coordinates_Imputed_Final.xlsx"
df.to_excel(output_path, index=False)

print(f"✅ Imputed file saved to: {output_path}")


✅ Imputed file saved to: Coordinates_Imputed_Final.xlsx
